# Binary text classification - IMDB - TransformerEncoder

In [1]:
# Load data and create datasets

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

# Listing 11.12 Preparing integer sequence datasets
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  31.5M      0  0:00:02  0:00:02 --:--:-- 31.5M
Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [2]:
# Listing 11.21 Transformer encoder implemented as a subclassed Layer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"),
         layers.Dense(embed_dim),])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)
  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,})
    return config

In [3]:
# isting 11.22 Using the Transformer encoder for text classification
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32
 
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [4]:
# Listing 11.23 Training and evaluating the Transformer encoder based model
callbacks = [
  keras.callbacks.ModelCheckpoint(
      "transformer_encoder.keras",
      save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
 callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 49s 68ms/step - loss: 0.4944 - accuracy: 0.7688 - val_loss: 0.3110 - val_accuracy: 0.8698
Epoch 2/20
625/625 [==============================] - 42s 67ms/step - loss: 0.3189 - accuracy: 0.8663 - val_loss: 0.3176 - val_accuracy: 0.8730
Epoch 3/20
625/625 [==============================] - 42s 67ms/step - loss: 0.2450 - accuracy: 0.9030 - val_loss: 0.2813 - val_accuracy: 0.8934
Epoch 4/20
625/625 [==============================] - 42s 68ms/step - loss: 0.1885 - accuracy: 0.9273 - val_loss: 0.4130 - val_accuracy: 0.8720
Epoch 5/20
625/625 [==============================] - 42s 68ms/step - loss: 0.1550 - accuracy: 0.9402 - val_loss: 0.3497 - val_accuracy: 0.8836
Epoch 6/20
625/625 [==============================] - 42s 67ms/step - loss: 0.1324 - accuracy: 0.9499 - val_loss: 0.4101 - val_accuracy: 0.8916
Epoch 7/20
625/625 [==============================] - 42s 68ms/step - loss: 0.1142 - accuracy: 0.9580 - val_loss: 0.3874 - val_accuracy:

In [5]:
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={
        "TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 19s 24ms/step - loss: 0.3151 - accuracy: 0.8759
Test acc: 0.876
